In [1]:
%matplotlib inline
from scipy.stats import randint as sp_randint
from scipy.stats import uniform
from scipy.stats import uniform as sp_randFloat
from sklearn import svm
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from time import time
import numpy as np
import pandas as pd
import sklearn
import warnings
warnings.filterwarnings('ignore')


In [2]:
from scipy.stats import randint as sp_randInt

from sklearn.model_selection import GridSearchCV, PredefinedSplit
from sklearn.metrics import make_scorer
from scipy import sparse

In [3]:
features= ['pck_size', 'Ether_type', 'LLC_ctrl', 'EAPOL_version', 'EAPOL_type', 'IP_ihl', 'IP_tos', 'IP_len', 'IP_flags', 'IP_DF', 'IP_ttl', 'IP_options', 'ICMP_code', 'TCP_dataofs', 'TCP_FIN', 'TCP_ACK', 'TCP_window', 'UDP_len', 'DHCP_options', 'BOOTP_hlen', 'BOOTP_flags', 'BOOTP_sname', 'BOOTP_file', 'BOOTP_options', 'DNS_qr', 'DNS_rd', 'DNS_qdcount', 'dport_class', 'payload_bytes', 'entropy', 'Label']

df=pd.read_csv("Aalto_train_IoTDevID.csv",usecols=features) 
X_train = df.iloc[:,0:-1]
df['Label'] = df['Label'].astype('category')
y_train=df['Label'].cat.codes  


df=pd.read_csv("Aalto_test_IoTDevID.csv",usecols=features) 
X_test = df.iloc[:,0:-1]
df['Label'] = df['Label'].astype('category')
y_test=df['Label'].cat.codes  

In [4]:
print(X_train.shape,X_test.shape)

(83894, 30) (21182, 30)


# Aalto

In [5]:


X= np.concatenate([X_train, X_test])
test_fold = [-1 for _ in range(X_train.shape[0])] + [0 for _ in range(X_test.shape[0])]
y = np.concatenate([y_train, y_test])
ps = PredefinedSplit(test_fold)

def run_random_search(model, params, x_train, y_train):
    #grid = GridSearchCV(model, params, cv = ps, n_jobs = -1, scoring = score, verbose = 0, refit = False)
    grid =RandomizedSearchCV(model, param_grid, cv=ps,scoring = 'f1_macro')
    grid.fit(x_train, y_train)
    return (grid.best_params_, round(grid.best_score_,8),grid.best_estimator_)

# RandomizedSearchCV  DT

In [11]:
print ('%-90s %-20s %-8s %-8s' % ("HYPERPARAMETERS","F1 Score", "Time", "No"))
  


nfolds=10
param_grid = { 'criterion':['gini','entropy'],
                  "max_depth":np.linspace(1, 32, 32, endpoint=True),
                 "min_samples_split": sp_randint(2,10),#uniform(0.1,1 ),
                    # "min_samples_leafs" : np.linspace(0.1, 0.5, 5, endpoint=True)
                    "max_features" : sp_randint(1,X_train.shape[1])}

second=time()
f1=[]
clf=DecisionTreeClassifier()
for ii in range(25):
    clf.fit(X_train, y_train)
    predict =clf.predict(X_test)
    f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
f1=sum(f1)/len(f1)   
#if f1>0.76:
print('%-90s %-20s %-8s %-8s' % ("default",f1,round(time()-second,3),ii))

for i in range(100):
    second=time()
    a,b,clf=run_random_search(DecisionTreeClassifier(),param_grid,X,y)
    f1=[]
    for ii in range(25):
        clf.fit(X_train, y_train)
        predict =clf.predict(X_test)
        f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
    f1=sum(f1)/len(f1)   
    #if f1>0.76:
    print('%-90s %-20s %-8s %-8s' % (a,f1,round(time()-second,3),i))
    
    
    
    


HYPERPARAMETERS                                                                            F1 Score             Time     No      
default                                                                                    0.7216903208093847   10.534   24      
{'criterion': 'gini', 'max_depth': 24.0, 'max_features': 29, 'min_samples_split': 2}       0.7270150996774428   14.381   0       
{'criterion': 'entropy', 'max_depth': 26.0, 'max_features': 29, 'min_samples_split': 9}    0.724980440036087    15.236   1       
{'criterion': 'gini', 'max_depth': 26.0, 'max_features': 13, 'min_samples_split': 2}       0.7252618968864211   9.218    2       
{'criterion': 'gini', 'max_depth': 30.0, 'max_features': 16, 'min_samples_split': 9}       0.7245593084832818   10.306   3       
{'criterion': 'gini', 'max_depth': 30.0, 'max_features': 27, 'min_samples_split': 6}       0.7251107950372347   13.949   4       
{'criterion': 'entropy', 'max_depth': 26.0, 'max_features': 20, 'min_samples_split': 9}   

{'criterion': 'gini', 'max_depth': 27.0, 'max_features': 21, 'min_samples_split': 4}       0.7252661448423163   11.706   62      
{'criterion': 'gini', 'max_depth': 20.0, 'max_features': 13, 'min_samples_split': 9}       0.7225537598391735   9.053    63      
{'criterion': 'gini', 'max_depth': 29.0, 'max_features': 27, 'min_samples_split': 5}       0.7248912901016741   13.058   64      
{'criterion': 'entropy', 'max_depth': 31.0, 'max_features': 23, 'min_samples_split': 6}    0.7222811428893577   12.971   65      
{'criterion': 'gini', 'max_depth': 25.0, 'max_features': 28, 'min_samples_split': 2}       0.7268905598898969   13.104   66      
{'criterion': 'gini', 'max_depth': 27.0, 'max_features': 18, 'min_samples_split': 9}       0.7244995620159433   10.208   67      
{'criterion': 'entropy', 'max_depth': 21.0, 'max_features': 21, 'min_samples_split': 2}    0.7239913735218103   11.833   68      
{'criterion': 'gini', 'max_depth': 22.0, 'max_features': 23, 'min_samples_split': 4}      

# GridSearchCV DT

In [21]:
param_grid = { 'criterion':['gini','entropy'],
                  "max_depth":list(range(1,32)),
                 "min_samples_split":list(range(2,10)),#uniform(0.1,1 ),
                    # "min_samples_leafs" : np.linspace(0.1, 0.5, 5, endpoint=True)
                    "max_features" :list(range(1,X_train.shape[1]))}



nbModel_grid = GridSearchCV(estimator=DecisionTreeClassifier(), param_grid=param_grid, verbose=1, cv=ps, n_jobs=-1)
nbModel_grid.fit(X, y)
print(nbModel_grid.best_estimator_)

Fitting 1 folds for each of 14384 candidates, totalling 14384 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    2.3s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:    7.9s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:   21.4s
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:   41.8s
[Parallel(n_jobs=-1)]: Done 1242 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 1792 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 2442 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done 3192 tasks      | elapsed:  4.2min
[Parallel(n_jobs=-1)]: Done 4042 tasks      | elapsed:  5.6min
[Parallel(n_jobs=-1)]: Done 4992 tasks      | elapsed:  7.3min
[Parallel(n_jobs=-1)]: Done 6042 tasks      | elapsed:  9.2min
[Parallel(n_jobs=-1)]: Done 7192 tasks      | elapsed: 11.3min
[Parallel(n_jobs=-1)]: Done 8442 tasks      | elapsed: 13.2min
[Parallel(n_jobs=-1)]: Done 9792 tasks      | elapsed: 16.3min
[Parallel(n_jobs=-1)]: Done 11242 tasks      |

DecisionTreeClassifier(criterion='entropy', max_depth=23, max_features=21)


# RandomizedSearchCV RF

In [12]:
# use a full grid over all parameters
param_grid = {"max_depth":np.linspace(1, 32, 32, endpoint=True),
              "n_estimators" : sp_randint(1, 200),
              "max_features": sp_randint(1, 11),
              "min_samples_split":sp_randint(2, 11),
              "bootstrap": [True, False],
              "criterion": ["gini", "entropy"]}
second=time()
f1=[]
clf=RandomForestClassifier()
for ii in range(1):
    clf.fit(X_train, y_train)
    predict =clf.predict(X_test)
    f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
f1=sum(f1)/len(f1)   
#if f1>0.76:
print('%-90s %-20s %-8s %-8s' % ("default",f1,round(time()-second,3),ii))

for i in range(50):
    second=time()
    a,b,clf=run_random_search(RandomForestClassifier(),param_grid,X,y)
    clf.fit(X_train, y_train)
    predict =clf.predict(X_test)
    f1=sklearn.metrics.f1_score(y_test, predict,average= "macro") 
    print('%-90s %-20s %-8s %-8s' % (a,f1,round(time()-second,3),i))

default                                                                                    0.7227299454671903   10.844   0       
{'bootstrap': True, 'criterion': 'entropy', 'max_depth': 22.0, 'max_features': 2, 'min_samples_split': 10, 'n_estimators': 156} 0.7289502435293613   159.548  0       
{'bootstrap': False, 'criterion': 'gini', 'max_depth': 25.0, 'max_features': 1, 'min_samples_split': 8, 'n_estimators': 113} 0.7262619535233411   193.489  1       
{'bootstrap': True, 'criterion': 'entropy', 'max_depth': 22.0, 'max_features': 10, 'min_samples_split': 10, 'n_estimators': 148} 0.7236580905885828   183.427  2       
{'bootstrap': True, 'criterion': 'entropy', 'max_depth': 27.0, 'max_features': 4, 'min_samples_split': 10, 'n_estimators': 82} 0.727171992889616    113.511  3       
{'bootstrap': True, 'criterion': 'entropy', 'max_depth': 29.0, 'max_features': 3, 'min_samples_split': 7, 'n_estimators': 90} 0.7246079997419235   147.829  4       
{'bootstrap': True, 'criterion': 'gini',

{'bootstrap': False, 'criterion': 'gini', 'max_depth': 19.0, 'max_features': 8, 'min_samples_split': 9, 'n_estimators': 159} 0.727076835646918    110.671  49      


# RandomizedSearchCV  KNeighborsClassifier

In [13]:
# use a full grid over all parameters
param_grid = {"n_neighbors" : sp_randint(1,64) ,  
             "leaf_size": sp_randint(1,50) , 
              "algorithm" : ["auto", "ball_tree", "kd_tree", "brute"],
              "weights" : ["uniform", "distance"]}
second=time()
f1=[]
clf=KNeighborsClassifier()
for ii in range(1):
    clf.fit(X_train, y_train)
    predict =clf.predict(X_test)
    f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
f1=sum(f1)/len(f1)   
#if f1>0.76:
print('%-90s %-20s %-8s %-8s' % ("default",f1,round(time()-second,3),i))


for i in range(50):
    second=time()
    a,b,clf=run_random_search(KNeighborsClassifier(),param_grid,X,y)

    clf.fit(X_train, y_train)
    predict =clf.predict(X_test)
    f1=sklearn.metrics.f1_score(y_test, predict,average= "macro") 
    print('%-90s %-20s %-8s %-8s' % (a,f1,round(time()-second,3),i))
    

default                                                                                    0.7097573078498426   31.042   49      
{'algorithm': 'auto', 'leaf_size': 30, 'n_neighbors': 63, 'weights': 'distance'}           0.7169344588030341   211.957  0       
{'algorithm': 'auto', 'leaf_size': 16, 'n_neighbors': 47, 'weights': 'distance'}           0.7176466921242463   245.564  1       
{'algorithm': 'brute', 'leaf_size': 18, 'n_neighbors': 44, 'weights': 'distance'}          0.7169670826387976   273.454  2       
{'algorithm': 'brute', 'leaf_size': 2, 'n_neighbors': 48, 'weights': 'distance'}           0.718017143478749    227.629  3       
{'algorithm': 'auto', 'leaf_size': 43, 'n_neighbors': 46, 'weights': 'distance'}           0.7176244604190383   228.35   4       
{'algorithm': 'auto', 'leaf_size': 18, 'n_neighbors': 40, 'weights': 'distance'}           0.7175984782540666   257.862  5       
{'algorithm': 'brute', 'leaf_size': 20, 'n_neighbors': 61, 'weights': 'distance'}         

# RandomizedSearchCV  GradientBoostingClassifier

In [14]:
# use a full grid over all parameters
param_grid =  {"learning_rate": sp_randFloat(),
          "subsample"    : sp_randFloat(),
          "n_estimators" : sp_randInt(100, 1000),
          "max_depth"    : sp_randInt(4, 10)
         }

second=time()
f1=[]
clf=GradientBoostingClassifier()
for ii in range(1):
    clf.fit(X_train, y_train)
    predict =clf.predict(X_test)
    f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
f1=sum(f1)/len(f1)   
#if f1>0.76:
print('%-90s %-20s %-8s %-8s' % ("default",f1,round(time()-second,3),ii))


for i in range(1):
    second=time()
    a,b,clf=run_random_search(GradientBoostingClassifier(),param_grid,X,y)
    clf.fit(X_train, y_train)
    predict =clf.predict(X_test)
    f1=sklearn.metrics.f1_score(y_test, predict,average= "macro") 
    print('%-90s %-20s %-8s %-8s' % (a,f1,round(time()-second,3),i))
    

default                                                                                    0.6247629147200783   323.341  0       
{'learning_rate': 0.1838641631843394, 'max_depth': 6, 'n_estimators': 535, 'subsample': 0.7134682210818548} 0.010075373269035157 41527.706 0       


#  SVM

In [15]:
param_grid =  {'C': [0.001, 0.01, 0.1, 1, 10], 'gamma' : [0.001, 0.01, 0.1, 1]}    
nbModel_grid = GridSearchCV(estimator=svm.SVC(), param_grid=param_grid, verbose=1, cv=ps, n_jobs=-1)
nbModel_grid.fit(X, y)
print(nbModel_grid.best_estimator_)    

Fitting 1 folds for each of 20 candidates, totalling 20 fits
SVC(C=10, gamma=1)


# RandomizedSearchCV  SVM

In [12]:
param_grid =  {'C': [0.001, 0.01, 0.1, 1, 10], 'gamma' : [0.001, 0.01, 0.1, 1]}  




second=time()
a,b,clf=run_random_search(svm.SVC(),param_grid,X,y)
clf.fit(X_train, y_train)
predict =clf.predict(X_test)
f1=(sklearn.metrics.f1_score(y_test, predict,average= "macro") )

print('%-90s %-20s %-8s %-8s' % (a,f1,round(time()-second,3),b))





{'gamma': 1, 'C': 10}                                                                      0.6492980530251848   6214.957 0.64929805


In [ ]:
param_grid =  {'C': [0.001, 0.01, 0.1, 1, 10], 'gamma' : [0.001, 0.01, 0.1, 1]}  



for i in range(33):
    second=time()
    a,b,clf=run_random_search(svm.SVC(),param_grid,X,y)
    f1=[]
    for ii in range(10):
        clf.fit(X_train, y_train)
        predict =clf.predict(X_test)
        f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
    f1=sum(f1)/len(f1)   
    #if f1>0.76:
    print('%-90s %-20s %-8s %-8s' % (a,f1,round(time()-second,3),i))

{'gamma': 1, 'C': 10}                                                                      0.649298053025185    10594.689 0       
{'gamma': 1, 'C': 1}                                                                       0.6346047487263045   9958.364 1       
{'gamma': 1, 'C': 10}                                                                      0.649298053025185    10890.422 2       
{'gamma': 0.1, 'C': 10}                                                                    0.6368980755609428   10017.98 3       
{'gamma': 1, 'C': 10}                                                                      0.649298053025185    9705.726 4       
{'gamma': 1, 'C': 10}                                                                      0.649298053025185    10802.529 5       
{'gamma': 1, 'C': 10}                                                                      0.649298053025185    10383.928 6       
{'gamma': 0.1, 'C': 1}                                                                

In [ ]:


param_grid = {"C": stats.uniform(0.001, 10),
             "gamma": stats.uniform(0.001, 1)}

for i in range(33):
    second=time()
    a,b,clf=run_random_search(svm.SVC(),param_grid,X,y)
    f1=[]
    for ii in range(10):
        clf.fit(X_train, y_train)
        predict =clf.predict(X_test)
        f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
    f1=sum(f1)/len(f1)   
    #if f1>0.76:
    print('%-90s %-20s %-8s %-8s' % (a,f1,round(time()-second,3),i))

#   NB

In [6]:
from sklearn.naive_bayes import CategoricalNB
from sklearn.model_selection import GridSearchCV

param_grid_nb = {
    'alpha': np.logspace(0,-9, num=100),
    "fit_prior":["True","False"]
}
nbModel_grid = GridSearchCV(estimator=CategoricalNB(), param_grid=param_grid_nb, verbose=1, cv=ps, n_jobs=-1)
nbModel_grid.fit(X, y)
print(nbModel_grid.best_estimator_)

Fitting 1 folds for each of 200 candidates, totalling 200 fits
CategoricalNB(alpha=1e-09, fit_prior='True')


In [10]:
from sklearn.naive_bayes import CategoricalNB
from sklearn.model_selection import GridSearchCV

param_grid_nb = {
    'alpha': np.logspace(0,-9, num=100),
    "fit_prior":["True","False"]
}
nbModel_grid = GridSearchCV(estimator=CategoricalNB(), param_grid=param_grid_nb, verbose=1, cv=ps, n_jobs=-1)
nbModel_grid.fit(X, y)
print(nbModel_grid.best_estimator_)

Fitting 1 folds for each of 200 candidates, totalling 200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   23.7s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 200 out of 200 | elapsed:  2.6min finished


CategoricalNB(alpha=1e-09, fit_prior='True')


# RandomizedSearchCV NB

In [11]:

second=time()

param_grid = {
    'alpha': np.logspace(0,-9, num=100),
    "fit_prior":["True","False"]
}

a,b,clf=run_random_search(CategoricalNB(),param_grid,X,y)
clf.fit(X_train, y_train)
predict =clf.predict(X_test)
f1=(sklearn.metrics.f1_score(y_test, predict,average= "macro") )

print('%-90s %-20s %-8s %-8s' % (a,f1,round(time()-second,3),b))



{'fit_prior': 'True', 'alpha': 1.873817422860383e-07}                                      0.5585787416692073   8.41     0.55857874


In [6]:
from sklearn.naive_bayes import CategoricalNB

In [8]:

second=time()

param_grid = {
    'alpha': np.logspace(0,-9, num=100),
    "fit_prior":["True","False"]
}

for i in range(100):
    second=time()
    a,b,clf=run_random_search(CategoricalNB(),param_grid,X,y)
    f1=[]
    for ii in range(25):
        clf.fit(X_train, y_train)
        predict =clf.predict(X_test)
        f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
    f1=sum(f1)/len(f1)   
    #if f1>0.76:
    print('%-90s %-20s %-8s %-8s' % (a,f1,round(time()-second,3),i))

{'fit_prior': 'False', 'alpha': 8.111308307896873e-08}                                     0.5585527392583353   27.113   0       
{'fit_prior': 'False', 'alpha': 5.336699231206313e-07}                                     0.5585321247002658   26.625   1       
{'fit_prior': 'False', 'alpha': 1.873817422860383e-08}                                     0.5585237755311087   27.33    2       
{'fit_prior': 'False', 'alpha': 2.310129700083158e-07}                                     0.5585787416692072   26.851   3       
{'fit_prior': 'False', 'alpha': 1.232846739442066e-07}                                     0.5585527392583353   28.996   4       
{'fit_prior': 'False', 'alpha': 5.336699231206313e-07}                                     0.5585321247002658   30.334   5       
{'fit_prior': 'False', 'alpha': 1.873817422860383e-07}                                     0.5585787416692072   25.975   6       
{'fit_prior': 'True', 'alpha': 2.848035868435799e-07}                                     

{'fit_prior': 'False', 'alpha': 1.232846739442066e-07}                                     0.5585527392583353   25.202   64      
{'fit_prior': 'False', 'alpha': 2.310129700083158e-07}                                     0.5585787416692072   29.14    65      
{'fit_prior': 'True', 'alpha': 2.310129700083158e-07}                                      0.5585787416692072   27.861   66      
{'fit_prior': 'False', 'alpha': 1.519911082952933e-09}                                     0.5585322432937763   25.329   67      
{'fit_prior': 'False', 'alpha': 1.232846739442066e-08}                                     0.5585237755311087   28.716   68      
{'fit_prior': 'False', 'alpha': 1e-07}                                                     0.5585527392583353   25.47    69      
{'fit_prior': 'False', 'alpha': 8.111308307896873e-08}                                     0.5585527392583353   26.286   70      
{'fit_prior': 'True', 'alpha': 1.519911082952933e-07}                                     

-------------

#  IoTSense- IoTsentinel 

In [1]:
%matplotlib inline
from scipy.stats import randint as sp_randint
from scipy.stats import uniform
from scipy.stats import uniform as sp_randFloat
from sklearn import svm
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from time import time
import numpy as np
import pandas as pd
import sklearn
import warnings
warnings.filterwarnings('ignore')


In [2]:
from scipy.stats import randint as sp_randInt

from sklearn.model_selection import GridSearchCV, PredefinedSplit
from sklearn.metrics import make_scorer
from scipy import sparse

# IoTSentinel

In [3]:
df=pd.read_csv("Aalto_IoTSentinel_Train.csv")
df

,ARP,LLC,EAPOL,IP,ICMP,ICMP6,TCP,UDP,HTTP,HTTPS,...,NTP,IP_padding,IP_add_count,IP_ralert,Portcl_src,Portcl_dst,Pck_size,Pck_rawdata,MAC,Label
0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,117,1,20:f8:5e:ca:91:52,Aria
1,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,95,1,20:f8:5e:ca:91:52,Aria
2,0,0,0,1,0,0,0,1,0,0,...,0,0,1,0,1,1,328,0,20:f8:5e:ca:91:52,Aria
3,0,0,0,1,0,0,0,1,0,0,...,0,0,1,0,1,1,328,0,20:f8:5e:ca:91:52,Aria
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,20:f8:5e:ca:91:52,Aria
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86079,0,0,0,1,0,0,1,0,1,0,...,0,0,3,0,3,1,313,1,00:24:e4:24:80:2a,Withings
86080,0,0,0,1,0,0,1,0,1,0,...,0,0,3,0,3,1,262,1,00:24:e4:24:80:2a,Withings
86081,0,0,0,1,0,0,1,0,1,0,...,0,0,3,0,3,1,40,0,00:24:e4:24:80:2a,Withings
86082,0,0,0,1,0,0,1,0,1,0,...,0,0,3,0,3,1,40,0,00:24:e4:24:80:2a,Withings


In [4]:
df.columns

Index(['ARP', 'LLC', 'EAPOL', 'IP', 'ICMP', 'ICMP6', 'TCP', 'UDP', 'HTTP',
       'HTTPS', 'DHCP', 'BOOTP', 'SSDP', 'DNS', 'MDNS', 'NTP', 'IP_padding',
       'IP_add_count', 'IP_ralert', 'Portcl_src', 'Portcl_dst', 'Pck_size',
       'Pck_rawdata', 'MAC', 'Label'],
      dtype='object')

In [5]:
features= ['ARP', 'LLC', 'EAPOL', 'IP', 'ICMP', 'ICMP6', 'TCP', 'UDP', 'HTTP',
       'HTTPS', 'DHCP', 'BOOTP', 'SSDP', 'DNS', 'MDNS', 'NTP', 'IP_padding',
       'IP_add_count', 'IP_ralert', 'Portcl_src', 'Portcl_dst', 'Pck_size',
       'Pck_rawdata',  'Label']
df=pd.read_csv("Aalto_IoTSentinel_Train.csv",usecols=features) 
X_train = df.iloc[:,0:-1]
df['Label'] = df['Label'].astype('category')
y_train=df['Label'].cat.codes  


df=pd.read_csv("Aalto_IoTSentinel_Test.csv",usecols=features) 
X_test = df.iloc[:,0:-1]
df['Label'] = df['Label'].astype('category')
y_test=df['Label'].cat.codes  

In [6]:
print(X_train.shape,X_test.shape)

(86084, 23) (21723, 23)


In [7]:
X= np.concatenate([X_train, X_test])
test_fold = [-1 for _ in range(X_train.shape[0])] + [0 for _ in range(X_test.shape[0])]
y = np.concatenate([y_train, y_test])
ps = PredefinedSplit(test_fold)

def run_random_search(model, params, x_train, y_train):
    #grid = GridSearchCV(model, params, cv = ps, n_jobs = -1, scoring = score, verbose = 0, refit = False)
    grid =RandomizedSearchCV(model, param_grid, cv=ps,scoring = 'f1_macro')
    grid.fit(x_train, y_train)
    return (grid.best_params_, round(grid.best_score_,8),grid.best_estimator_)

# RandomizedSearchCV  DT

In [8]:
print ('%-90s %-20s %-8s %-8s' % ("HYPERPARAMETERS","F1 Score", "Time", "No"))
  


nfolds=10
param_grid = { 'criterion':['gini','entropy'],
                  "max_depth":np.linspace(1, 32, 32, endpoint=True),
                 "min_samples_split": sp_randint(2,10),#uniform(0.1,1 ),
                    # "min_samples_leafs" : np.linspace(0.1, 0.5, 5, endpoint=True)
                    "max_features" : sp_randint(1,X_train.shape[1])}

second=time()
f1=[]
clf=DecisionTreeClassifier()
for ii in range(25):
    clf.fit(X_train, y_train)
    predict =clf.predict(X_test)
    f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
f1=sum(f1)/len(f1)   
#if f1>0.76:
print('%-90s %-20s %-8s %-8s' % ("default",f1,round(time()-second,3),ii))

for i in range(100):
    second=time()
    a,b,clf=run_random_search(DecisionTreeClassifier(),param_grid,X,y)
    f1=[]
    for ii in range(25):
        clf.fit(X_train, y_train)
        predict =clf.predict(X_test)
        f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
    f1=sum(f1)/len(f1)   
    #if f1>0.76:
    print('%-90s %-20s %-8s %-8s' % (a,f1,round(time()-second,3),i))
    
    
    
    


HYPERPARAMETERS                                                                            F1 Score             Time     No      
default                                                                                    0.6009580335884174   3.297    24      
{'criterion': 'entropy', 'max_depth': 29.0, 'max_features': 19, 'min_samples_split': 6}    0.6030508357696247   3.882    0       
{'criterion': 'entropy', 'max_depth': 28.0, 'max_features': 11, 'min_samples_split': 2}    0.6019134446933325   2.7      1       
{'criterion': 'entropy', 'max_depth': 30.0, 'max_features': 7, 'min_samples_split': 4}     0.6008899861597053   2.675    2       
{'criterion': 'gini', 'max_depth': 25.0, 'max_features': 12, 'min_samples_split': 2}       0.6011518449043427   3.048    3       
{'criterion': 'gini', 'max_depth': 29.0, 'max_features': 14, 'min_samples_split': 4}       0.6019271172312137   3.166    4       
{'criterion': 'entropy', 'max_depth': 21.0, 'max_features': 8, 'min_samples_split': 5}    

{'criterion': 'entropy', 'max_depth': 30.0, 'max_features': 16, 'min_samples_split': 8}    0.6014832428943402   3.58     62      
{'criterion': 'entropy', 'max_depth': 20.0, 'max_features': 4, 'min_samples_split': 6}     0.6023492070265482   2.658    63      
{'criterion': 'entropy', 'max_depth': 31.0, 'max_features': 14, 'min_samples_split': 6}    0.6031510585593872   3.392    64      
{'criterion': 'entropy', 'max_depth': 30.0, 'max_features': 6, 'min_samples_split': 3}     0.601555484339515    2.832    65      
{'criterion': 'entropy', 'max_depth': 31.0, 'max_features': 2, 'min_samples_split': 4}     0.601422434493549    2.776    66      
{'criterion': 'gini', 'max_depth': 25.0, 'max_features': 16, 'min_samples_split': 5}       0.6020133204779852   3.343    67      
{'criterion': 'entropy', 'max_depth': 17.0, 'max_features': 14, 'min_samples_split': 6}    0.599879166738513    3.176    68      
{'criterion': 'gini', 'max_depth': 20.0, 'max_features': 16, 'min_samples_split': 6}      

# IoT Sense

In [13]:
df=pd.read_csv("Aalto_IoTSense_Train.csv")
df

,ARP,EAPOL,IP,ICMP,ICMP6,TCP,UDP,TCP_w_size,HTTP,HTTPS,...,SSDP,DNS,MDNS,NTP,IP_padding,IP_ralert,payload_l,Entropy,MAC,Label
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,4.175388,20:f8:5e:ca:91:52,Aria
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1.610262,20:f8:5e:ca:91:52,Aria
2,0,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,300,0.000000,20:f8:5e:ca:91:52,Aria
3,0,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,300,0.000000,20:f8:5e:ca:91:52,Aria
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.000000,20:f8:5e:ca:91:52,Aria
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86079,0,0,1,0,0,1,0,8400,1,0,...,0,0,0,0,0,0,273,5.144670,00:24:e4:24:80:2a,Withings
86080,0,0,1,0,0,1,0,7991,1,0,...,0,0,0,0,0,0,222,5.123301,00:24:e4:24:80:2a,Withings
86081,0,0,1,0,0,1,0,7614,1,0,...,0,0,0,0,0,0,0,0.000000,00:24:e4:24:80:2a,Withings
86082,0,0,1,0,0,1,0,7614,1,0,...,0,0,0,0,0,0,0,0.000000,00:24:e4:24:80:2a,Withings


In [14]:
df.columns

Index(['ARP', 'EAPOL', 'IP', 'ICMP', 'ICMP6', 'TCP', 'UDP', 'TCP_w_size',
       'HTTP', 'HTTPS', 'DHCP', 'BOOTP', 'SSDP', 'DNS', 'MDNS', 'NTP',
       'IP_padding', 'IP_ralert', 'payload_l', 'Entropy', 'MAC', 'Label'],
      dtype='object')

In [15]:
features= ['ARP', 'EAPOL', 'IP', 'ICMP', 'ICMP6', 'TCP', 'UDP', 'TCP_w_size',
       'HTTP', 'HTTPS', 'DHCP', 'BOOTP', 'SSDP', 'DNS', 'MDNS', 'NTP',
       'IP_padding', 'IP_ralert', 'payload_l', 'Entropy', 'Label']
df=pd.read_csv("Aalto_IoTSense_Train.csv",usecols=features) 
X_train = df.iloc[:,0:-1]
df['Label'] = df['Label'].astype('category')
y_train=df['Label'].cat.codes  


df=pd.read_csv("Aalto_IoTSense_Test.csv",usecols=features) 
X_test = df.iloc[:,0:-1]
df['Label'] = df['Label'].astype('category')
y_test=df['Label'].cat.codes  

In [16]:
print(X_train.shape,X_test.shape)

(86084, 20) (21723, 20)


In [17]:
X= np.concatenate([X_train, X_test])
test_fold = [-1 for _ in range(X_train.shape[0])] + [0 for _ in range(X_test.shape[0])]
y = np.concatenate([y_train, y_test])
ps = PredefinedSplit(test_fold)

def run_random_search(model, params, x_train, y_train):
    #grid = GridSearchCV(model, params, cv = ps, n_jobs = -1, scoring = score, verbose = 0, refit = False)
    grid =RandomizedSearchCV(model, param_grid, cv=ps,scoring = 'f1_macro')
    grid.fit(x_train, y_train)
    return (grid.best_params_, round(grid.best_score_,8),grid.best_estimator_)

# RandomizedSearchCV  DT

In [18]:
print ('%-90s %-20s %-8s %-8s' % ("HYPERPARAMETERS","F1 Score", "Time", "No"))
  


nfolds=10
param_grid = { 'criterion':['gini','entropy'],
                  "max_depth":np.linspace(1, 32, 32, endpoint=True),
                 "min_samples_split": sp_randint(2,10),#uniform(0.1,1 ),
                    # "min_samples_leafs" : np.linspace(0.1, 0.5, 5, endpoint=True)
                    "max_features" : sp_randint(1,X_train.shape[1])}

second=time()
f1=[]
clf=DecisionTreeClassifier()
for ii in range(25):
    clf.fit(X_train, y_train)
    predict =clf.predict(X_test)
    f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
f1=sum(f1)/len(f1)   
#if f1>0.76:
print('%-90s %-20s %-8s %-8s' % ("default",f1,round(time()-second,3),ii))

for i in range(100):
    second=time()
    a,b,clf=run_random_search(DecisionTreeClassifier(),param_grid,X,y)
    f1=[]
    for ii in range(25):
        clf.fit(X_train, y_train)
        predict =clf.predict(X_test)
        f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
    f1=sum(f1)/len(f1)   
    #if f1>0.76:
    print('%-90s %-20s %-8s %-8s' % (a,f1,round(time()-second,3),i))
    
    
    
    


HYPERPARAMETERS                                                                            F1 Score             Time     No      
default                                                                                    0.558691645008419    3.763    24      
{'criterion': 'entropy', 'max_depth': 29.0, 'max_features': 15, 'min_samples_split': 2}    0.5595939405707852   5.056    0       
{'criterion': 'gini', 'max_depth': 27.0, 'max_features': 17, 'min_samples_split': 5}       0.5598395436693392   4.321    1       
{'criterion': 'gini', 'max_depth': 27.0, 'max_features': 11, 'min_samples_split': 5}       0.5588452549829972   3.867    2       
{'criterion': 'gini', 'max_depth': 30.0, 'max_features': 9, 'min_samples_split': 2}        0.558784869621737    4.126    3       
{'criterion': 'gini', 'max_depth': 22.0, 'max_features': 10, 'min_samples_split': 7}       0.5584838920603935   3.171    4       
{'criterion': 'entropy', 'max_depth': 26.0, 'max_features': 17, 'min_samples_split': 3}   

{'criterion': 'entropy', 'max_depth': 20.0, 'max_features': 19, 'min_samples_split': 6}    0.5607282578231187   5.315    62      
{'criterion': 'entropy', 'max_depth': 25.0, 'max_features': 5, 'min_samples_split': 2}     0.5580912023897762   3.76     63      
{'criterion': 'gini', 'max_depth': 29.0, 'max_features': 17, 'min_samples_split': 4}       0.5587925475051779   4.098    64      
{'criterion': 'entropy', 'max_depth': 21.0, 'max_features': 11, 'min_samples_split': 6}    0.5588781392163361   4.092    65      
{'criterion': 'entropy', 'max_depth': 25.0, 'max_features': 18, 'min_samples_split': 7}    0.5602472665899518   4.968    66      
{'criterion': 'gini', 'max_depth': 25.0, 'max_features': 15, 'min_samples_split': 2}       0.55939776875985     3.675    67      
{'criterion': 'gini', 'max_depth': 22.0, 'max_features': 13, 'min_samples_split': 6}       0.5596147153375949   3.511    68      
{'criterion': 'gini', 'max_depth': 31.0, 'max_features': 17, 'min_samples_split': 2}      

________________

# UNSW

In [3]:
%matplotlib inline
from scipy.stats import randint as sp_randint
from scipy.stats import uniform
from scipy.stats import uniform as sp_randFloat
from sklearn import svm
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from time import time
import numpy as np
import pandas as pd
import sklearn
import warnings
warnings.filterwarnings('ignore')


In [4]:
from scipy.stats import randint as sp_randInt

from sklearn.model_selection import GridSearchCV, PredefinedSplit
from sklearn.metrics import make_scorer
from scipy import sparse

# IoTSentinel

In [5]:
df=pd.read_csv("UNSW_IoTSentinel_Train.csv")
df

,ARP,LLC,EAPOL,IP,ICMP,ICMP6,TCP,UDP,HTTP,HTTPS,...,NTP,IP_padding,IP_add_count,IP_ralert,Portcl_src,Portcl_dst,Pck_size,Pck_rawdata,MAC,Label
0,0,0,0,1,0,0,0,1,0,0,...,1,0,37,0,2,1,76,0,44:65:0d:56:cc:d3,Amazon Echo
1,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,95,1,44:65:0d:56:cc:d3,Amazon Echo
2,0,0,0,1,0,0,1,0,1,0,...,0,0,31,0,3,1,60,0,44:65:0d:56:cc:d3,Amazon Echo
3,0,0,0,1,0,0,0,1,0,0,...,0,0,37,0,2,2,177,1,44:65:0d:56:cc:d3,Amazon Echo
4,0,0,0,1,0,0,0,1,0,0,...,0,0,36,0,3,2,187,1,44:65:0d:56:cc:d3,Amazon Echo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
282806,0,0,0,1,0,0,1,0,1,0,...,0,0,4,0,3,1,40,0,00:24:e4:1b:6f:96,Withings Smart scale
282807,0,0,0,1,0,0,1,0,1,0,...,0,0,4,0,3,1,262,1,00:24:e4:1b:6f:96,Withings Smart scale
282808,0,0,0,1,0,0,1,0,1,0,...,0,0,4,0,3,1,40,0,00:24:e4:1b:6f:96,Withings Smart scale
282809,0,0,0,1,0,0,1,0,1,0,...,0,0,4,0,3,1,40,0,00:24:e4:1b:6f:96,Withings Smart scale


In [6]:
df.columns

Index(['ARP', 'LLC', 'EAPOL', 'IP', 'ICMP', 'ICMP6', 'TCP', 'UDP', 'HTTP',
       'HTTPS', 'DHCP', 'BOOTP', 'SSDP', 'DNS', 'MDNS', 'NTP', 'IP_padding',
       'IP_add_count', 'IP_ralert', 'Portcl_src', 'Portcl_dst', 'Pck_size',
       'Pck_rawdata', 'MAC', 'Label'],
      dtype='object')

In [7]:
features= ['ARP', 'LLC', 'EAPOL', 'IP', 'ICMP', 'ICMP6', 'TCP', 'UDP', 'HTTP',
       'HTTPS', 'DHCP', 'BOOTP', 'SSDP', 'DNS', 'MDNS', 'NTP', 'IP_padding',
       'IP_add_count', 'IP_ralert', 'Portcl_src', 'Portcl_dst', 'Pck_size',
       'Pck_rawdata',  'Label']
df=pd.read_csv("UNSW_IoTSentinel_Train.csv",usecols=features) 
X_train = df.iloc[:,0:-1]
df['Label'] = df['Label'].astype('category')
y_train=df['Label'].cat.codes  


df=pd.read_csv("UNSW_IoTSentinel_Test.csv",usecols=features) 
X_test = df.iloc[:,0:-1]
df['Label'] = df['Label'].astype('category')
y_test=df['Label'].cat.codes  

In [8]:
print(X_train.shape,X_test.shape)

(282811, 23) (70449, 23)


In [9]:
X= np.concatenate([X_train, X_test])
test_fold = [-1 for _ in range(X_train.shape[0])] + [0 for _ in range(X_test.shape[0])]
y = np.concatenate([y_train, y_test])
ps = PredefinedSplit(test_fold)

def run_random_search(model, params, x_train, y_train):
    #grid = GridSearchCV(model, params, cv = ps, n_jobs = -1, scoring = score, verbose = 0, refit = False)
    grid =RandomizedSearchCV(model, param_grid, cv=ps,scoring = 'f1_macro')
    grid.fit(x_train, y_train)
    return (grid.best_params_, round(grid.best_score_,8),grid.best_estimator_)

# RandomizedSearchCV  DT

In [10]:
print ('%-90s %-20s %-8s %-8s' % ("HYPERPARAMETERS","F1 Score", "Time", "No"))
  


nfolds=10
param_grid = { 'criterion':['gini','entropy'],
                  "max_depth":np.linspace(1, 32, 32, endpoint=True),
                 "min_samples_split": sp_randint(2,10),#uniform(0.1,1 ),
                    # "min_samples_leafs" : np.linspace(0.1, 0.5, 5, endpoint=True)
                    "max_features" : sp_randint(1,X_train.shape[1])}

second=time()
f1=[]
clf=DecisionTreeClassifier()
for ii in range(25):
    clf.fit(X_train, y_train)
    predict =clf.predict(X_test)
    f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
f1=sum(f1)/len(f1)   
#if f1>0.76:
print('%-90s %-20s %-8s %-8s' % ("default",f1,round(time()-second,3),ii))

for i in range(100):
    second=time()
    a,b,clf=run_random_search(DecisionTreeClassifier(),param_grid,X,y)
    f1=[]
    for ii in range(25):
        clf.fit(X_train, y_train)
        predict =clf.predict(X_test)
        f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
    f1=sum(f1)/len(f1)   
    #if f1>0.76:
    print('%-90s %-20s %-8s %-8s' % (a,f1,round(time()-second,3),i))
    
    
    
    


HYPERPARAMETERS                                                                            F1 Score             Time     No      
default                                                                                    0.504726029658401    13.599   24      
{'criterion': 'entropy', 'max_depth': 27.0, 'max_features': 6, 'min_samples_split': 5}     0.5011076161175921   10.733   0       
{'criterion': 'gini', 'max_depth': 31.0, 'max_features': 1, 'min_samples_split': 4}        0.5099407835954021   10.322   1       
{'criterion': 'gini', 'max_depth': 27.0, 'max_features': 20, 'min_samples_split': 7}       0.5034832751111291   17.245   2       
{'criterion': 'gini', 'max_depth': 14.0, 'max_features': 8, 'min_samples_split': 8}        0.4840597514969355   11.02    3       
{'criterion': 'gini', 'max_depth': 16.0, 'max_features': 8, 'min_samples_split': 9}        0.4902067490058035   12.682   4       
{'criterion': 'entropy', 'max_depth': 20.0, 'max_features': 12, 'min_samples_split': 2}   

{'criterion': 'entropy', 'max_depth': 21.0, 'max_features': 18, 'min_samples_split': 5}    0.5001553785231405   17.376   62      
{'criterion': 'gini', 'max_depth': 30.0, 'max_features': 11, 'min_samples_split': 6}       0.5048528597366588   15.338   63      
{'criterion': 'entropy', 'max_depth': 25.0, 'max_features': 7, 'min_samples_split': 8}     0.5031902178663394   14.358   64      
{'criterion': 'gini', 'max_depth': 27.0, 'max_features': 15, 'min_samples_split': 6}       0.5068745596054003   17.978   65      
{'criterion': 'gini', 'max_depth': 28.0, 'max_features': 17, 'min_samples_split': 2}       0.5071040567497408   20.052   66      
{'criterion': 'entropy', 'max_depth': 16.0, 'max_features': 3, 'min_samples_split': 6}     0.502716077553813    13.01    67      
{'criterion': 'gini', 'max_depth': 20.0, 'max_features': 22, 'min_samples_split': 6}       0.49997792082799825  23.418   68      
{'criterion': 'gini', 'max_depth': 31.0, 'max_features': 5, 'min_samples_split': 7}       

# IoT Sense

In [11]:
df=pd.read_csv("UNSW_IoTSense_Train.csv")
df

,ARP,EAPOL,IP,ICMP,ICMP6,TCP,UDP,TCP_w_size,HTTP,HTTPS,...,SSDP,DNS,MDNS,NTP,IP_padding,IP_ralert,payload_l,Entropy,MAC,Label
0,0,0,1,0,0,0,1,0,0,0,...,0,0,0,1,0,0,48,1.290573,44:65:0d:56:cc:d3,Amazon Echo
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1.711966,44:65:0d:56:cc:d3,Amazon Echo
2,0,0,1,0,0,1,0,4380,1,0,...,0,0,0,0,0,0,0,0.000000,44:65:0d:56:cc:d3,Amazon Echo
3,0,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,149,5.710742,44:65:0d:56:cc:d3,Amazon Echo
4,0,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,159,5.696487,44:65:0d:56:cc:d3,Amazon Echo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
282806,0,0,1,0,0,1,0,8014,1,0,...,0,0,0,0,0,0,0,0.000000,00:24:e4:1b:6f:96,Withings Smart scale
282807,0,0,1,0,0,1,0,8014,1,0,...,0,0,0,0,0,0,222,5.120422,00:24:e4:1b:6f:96,Withings Smart scale
282808,0,0,1,0,0,1,0,7637,1,0,...,0,0,0,0,0,0,0,0.000000,00:24:e4:1b:6f:96,Withings Smart scale
282809,0,0,1,0,0,1,0,7637,1,0,...,0,0,0,0,0,0,0,0.000000,00:24:e4:1b:6f:96,Withings Smart scale


In [12]:
df.columns

Index(['ARP', 'EAPOL', 'IP', 'ICMP', 'ICMP6', 'TCP', 'UDP', 'TCP_w_size',
       'HTTP', 'HTTPS', 'DHCP', 'BOOTP', 'SSDP', 'DNS', 'MDNS', 'NTP',
       'IP_padding', 'IP_ralert', 'payload_l', 'Entropy', 'MAC', 'Label'],
      dtype='object')

In [17]:
features= ['ARP', 'EAPOL', 'IP', 'ICMP', 'ICMP6', 'TCP', 'UDP', 'TCP_w_size',
       'HTTP', 'HTTPS', 'DHCP', 'BOOTP', 'SSDP', 'DNS', 'MDNS', 'NTP',
       'IP_padding', 'IP_ralert', 'payload_l', 'Entropy', 'Label']
df=pd.read_csv("UNSW_IoTSense_Train.csv",usecols=features) 
X_train = df.iloc[:,0:-1]
df['Label'] = df['Label'].astype('category')
y_train=df['Label'].cat.codes  


df=pd.read_csv("UNSW_IoTSense_Test.csv",usecols=features) 
X_test = df.iloc[:,0:-1]
df['Label'] = df['Label'].astype('category')
y_test=df['Label'].cat.codes  

In [18]:
print(X_train.shape,X_test.shape)

(282811, 20) (70449, 20)


In [19]:
X= np.concatenate([X_train, X_test])
test_fold = [-1 for _ in range(X_train.shape[0])] + [0 for _ in range(X_test.shape[0])]
y = np.concatenate([y_train, y_test])
ps = PredefinedSplit(test_fold)

def run_random_search(model, params, x_train, y_train):
    #grid = GridSearchCV(model, params, cv = ps, n_jobs = -1, scoring = score, verbose = 0, refit = False)
    grid =RandomizedSearchCV(model, param_grid, cv=ps,scoring = 'f1_macro')
    grid.fit(x_train, y_train)
    return (grid.best_params_, round(grid.best_score_,8),grid.best_estimator_)

# RandomizedSearchCV  DT

In [20]:
print ('%-90s %-20s %-8s %-8s' % ("HYPERPARAMETERS","F1 Score", "Time", "No"))
  


nfolds=10
param_grid = { 'criterion':['gini','entropy'],
                  "max_depth":np.linspace(1, 32, 32, endpoint=True),
                 "min_samples_split": sp_randint(2,10),#uniform(0.1,1 ),
                    # "min_samples_leafs" : np.linspace(0.1, 0.5, 5, endpoint=True)
                    "max_features" : sp_randint(1,X_train.shape[1])}

second=time()
f1=[]
clf=DecisionTreeClassifier()
for ii in range(25):
    clf.fit(X_train, y_train)
    predict =clf.predict(X_test)
    f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
f1=sum(f1)/len(f1)   
#if f1>0.76:
print('%-90s %-20s %-8s %-8s' % ("default",f1,round(time()-second,3),ii))

for i in range(100):
    second=time()
    a,b,clf=run_random_search(DecisionTreeClassifier(),param_grid,X,y)
    f1=[]
    for ii in range(25):
        clf.fit(X_train, y_train)
        predict =clf.predict(X_test)
        f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
    f1=sum(f1)/len(f1)   
    #if f1>0.76:
    print('%-90s %-20s %-8s %-8s' % (a,f1,round(time()-second,3),i))
    
    
    
    


HYPERPARAMETERS                                                                            F1 Score             Time     No      
default                                                                                    0.7001257412876442   12.571   24      
{'criterion': 'entropy', 'max_depth': 26.0, 'max_features': 10, 'min_samples_split': 2}    0.6962463666427438   16.541   0       
{'criterion': 'gini', 'max_depth': 31.0, 'max_features': 16, 'min_samples_split': 6}       0.6838823911295954   18.952   1       
{'criterion': 'entropy', 'max_depth': 29.0, 'max_features': 13, 'min_samples_split': 4}    0.6936496038475042   18.467   2       
{'criterion': 'entropy', 'max_depth': 20.0, 'max_features': 6, 'min_samples_split': 4}     0.6912024571500703   14.505   3       
{'criterion': 'gini', 'max_depth': 32.0, 'max_features': 5, 'min_samples_split': 3}        0.6893947984204619   12.652   4       
{'criterion': 'gini', 'max_depth': 24.0, 'max_features': 6, 'min_samples_split': 2}       

{'criterion': 'entropy', 'max_depth': 27.0, 'max_features': 1, 'min_samples_split': 8}     0.6845832506103366   12.351   62      
{'criterion': 'entropy', 'max_depth': 29.0, 'max_features': 17, 'min_samples_split': 2}    0.6979122231588134   19.791   63      
{'criterion': 'entropy', 'max_depth': 27.0, 'max_features': 11, 'min_samples_split': 2}    0.7020353530806149   17.137   64      
{'criterion': 'entropy', 'max_depth': 18.0, 'max_features': 4, 'min_samples_split': 2}     0.6868099548358466   13.787   65      
{'criterion': 'gini', 'max_depth': 28.0, 'max_features': 13, 'min_samples_split': 3}       0.688253723725793    15.54    66      
{'criterion': 'gini', 'max_depth': 21.0, 'max_features': 13, 'min_samples_split': 8}       0.6806916351085426   15.052   67      
{'criterion': 'entropy', 'max_depth': 31.0, 'max_features': 5, 'min_samples_split': 8}     0.6890773947162215   13.896   68      
{'criterion': 'entropy', 'max_depth': 23.0, 'max_features': 10, 'min_samples_split': 2}   

# IoTDevID

In [21]:
df=pd.read_csv("UNSW_train_IoTDevID.csv")
df

,pck_size,Ether_type,LLC_dsap,LLC_ssap,LLC_ctrl,EAPOL_version,EAPOL_type,EAPOL_len,IP_version,IP_ihl,...,dport_bare,TCP_sport,TCP_dport,UDP_sport,UDP_dport,payload_bytes,payload_entropy,MAC,Protocol,Label
0,76,2048,0,0,0,0,0,0,4,5,...,123,0,0,33247,123,48,1.290573,44:65:0d:56:cc:d3,10,Amazon Echo
1,95,34958,0,0,0,1,3,95,0,0,...,0,0,0,0,0,95,1.711966,44:65:0d:56:cc:d3,1,Amazon Echo
2,60,2048,0,0,0,0,0,0,4,5,...,80,53487,80,0,0,0,0.000000,44:65:0d:56:cc:d3,4,Amazon Echo
3,177,2048,0,0,0,0,0,0,4,5,...,33434,0,0,47532,33434,149,5.710742,44:65:0d:56:cc:d3,12,Amazon Echo
4,187,2048,0,0,0,0,0,0,4,5,...,33434,0,0,49280,33434,159,5.696487,44:65:0d:56:cc:d3,12,Amazon Echo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
282806,40,2048,0,0,0,0,0,0,4,5,...,80,49153,80,0,0,0,0.000000,00:24:e4:1b:6f:96,4,Withings Smart scale
282807,262,2048,0,0,0,0,0,0,4,5,...,80,49153,80,0,0,222,5.120422,00:24:e4:1b:6f:96,5,Withings Smart scale
282808,40,2048,0,0,0,0,0,0,4,5,...,80,49153,80,0,0,0,0.000000,00:24:e4:1b:6f:96,4,Withings Smart scale
282809,40,2048,0,0,0,0,0,0,4,5,...,80,49153,80,0,0,0,0.000000,00:24:e4:1b:6f:96,4,Withings Smart scale


In [22]:
df.columns

Index(['pck_size', 'Ether_type', 'LLC_dsap', 'LLC_ssap', 'LLC_ctrl',
       'EAPOL_version', 'EAPOL_type', 'EAPOL_len', 'IP_version', 'IP_ihl',
       'IP_tos', 'IP_len', 'IP_flags', 'IP_Z', 'IP_MF', 'IP_id', 'IP_chksum',
       'IP_DF', 'IP_frag', 'IP_ttl', 'IP_proto', 'IP_options', 'IP_add_count',
       'ICMP_type', 'ICMP_code', 'ICMP_chksum', 'ICMP_id', 'ICMP_seq',
       'ICMP_ts_ori', 'ICMP_ts_rx', 'ICMP_ts_tx', 'ICMP_ptr', 'ICMP_reserved',
       'ICMP_length', 'ICMP_nexthopmtu', 'ICMP_unused', 'TCP_seq', 'TCP_ack',
       'TCP_dataofs', 'TCP_reserved', 'TCP_flags', 'TCP_FIN', 'TCP_SYN',
       'TCP_RST', 'TCP_PSH', 'TCP_ACK', 'TCP_URG', 'TCP_ECE', 'TCP_CWR',
       'TCP_window', 'TCP_chksum', 'TCP_urgptr', 'TCP_options', 'UDP_len',
       'UDP_chksum', 'DHCP_options', 'BOOTP_op', 'BOOTP_htype', 'BOOTP_hlen',
       'BOOTP_hops', 'BOOTP_xid', 'BOOTP_secs', 'BOOTP_flags', 'BOOTP_sname',
       'BOOTP_file', 'BOOTP_options', 'DNS_length', 'DNS_id', 'DNS_qr',
       'DNS_opcode', '

In [23]:
features= ['pck_size', 'Ether_type', 'LLC_ctrl', 'EAPOL_version', 'EAPOL_type', 'IP_ihl', 'IP_tos', 'IP_len', 'IP_flags', 'IP_DF', 'IP_ttl', 'IP_options', 'ICMP_code', 'TCP_dataofs', 'TCP_FIN', 'TCP_ACK', 'TCP_window', 'UDP_len', 'DHCP_options', 'BOOTP_hlen', 'BOOTP_flags', 'BOOTP_sname', 'BOOTP_file', 'BOOTP_options', 'DNS_qr', 'DNS_rd', 'DNS_qdcount', 'dport_class', 'payload_bytes', 'entropy',
'Label']
df=pd.read_csv("UNSW_train_IoTDevID.csv",usecols=features) 
X_train = df.iloc[:,0:-1]
df['Label'] = df['Label'].astype('category')
y_train=df['Label'].cat.codes  


df=pd.read_csv("UNSW_test_IoTDevID.csv",usecols=features) 
X_test = df.iloc[:,0:-1]
df['Label'] = df['Label'].astype('category')
y_test=df['Label'].cat.codes  

In [24]:
print(X_train.shape,X_test.shape)

(282811, 30) (70449, 30)


In [25]:
X= np.concatenate([X_train, X_test])
test_fold = [-1 for _ in range(X_train.shape[0])] + [0 for _ in range(X_test.shape[0])]
y = np.concatenate([y_train, y_test])
ps = PredefinedSplit(test_fold)

def run_random_search(model, params, x_train, y_train):
    #grid = GridSearchCV(model, params, cv = ps, n_jobs = -1, scoring = score, verbose = 0, refit = False)
    grid =RandomizedSearchCV(model, param_grid, cv=ps,scoring = 'f1_macro')
    grid.fit(x_train, y_train)
    return (grid.best_params_, round(grid.best_score_,8),grid.best_estimator_)

# RandomizedSearchCV  DT

In [26]:
print ('%-90s %-20s %-8s %-8s' % ("HYPERPARAMETERS","F1 Score", "Time", "No"))
  


nfolds=10
param_grid = { 'criterion':['gini','entropy'],
                  "max_depth":np.linspace(1, 32, 32, endpoint=True),
                 "min_samples_split": sp_randint(2,10),#uniform(0.1,1 ),
                    # "min_samples_leafs" : np.linspace(0.1, 0.5, 5, endpoint=True)
                    "max_features" : sp_randint(1,X_train.shape[1])}

second=time()
f1=[]
clf=DecisionTreeClassifier()
for ii in range(25):
    clf.fit(X_train, y_train)
    predict =clf.predict(X_test)
    f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
f1=sum(f1)/len(f1)   
#if f1>0.76:
print('%-90s %-20s %-8s %-8s' % ("default",f1,round(time()-second,3),ii))

for i in range(100):
    second=time()
    a,b,clf=run_random_search(DecisionTreeClassifier(),param_grid,X,y)
    f1=[]
    for ii in range(25):
        clf.fit(X_train, y_train)
        predict =clf.predict(X_test)
        f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
    f1=sum(f1)/len(f1)   
    #if f1>0.76:
    print('%-90s %-20s %-8s %-8s' % (a,f1,round(time()-second,3),i))
    
    
    
    


HYPERPARAMETERS                                                                            F1 Score             Time     No      
default                                                                                    0.8195978420312032   21.566   24      
{'criterion': 'entropy', 'max_depth': 30.0, 'max_features': 5, 'min_samples_split': 7}     0.8323455036302395   14.317   0       
{'criterion': 'gini', 'max_depth': 28.0, 'max_features': 9, 'min_samples_split': 4}        0.8280182896840249   16.834   1       
{'criterion': 'gini', 'max_depth': 22.0, 'max_features': 22, 'min_samples_split': 7}       0.8234451217809853   21.915   2       
{'criterion': 'entropy', 'max_depth': 27.0, 'max_features': 17, 'min_samples_split': 8}    0.8401239596610508   22.14    3       
{'criterion': 'entropy', 'max_depth': 30.0, 'max_features': 17, 'min_samples_split': 3}    0.8389146492593934   21.593   4       
{'criterion': 'entropy', 'max_depth': 26.0, 'max_features': 24, 'min_samples_split': 3}   

{'criterion': 'entropy', 'max_depth': 19.0, 'max_features': 10, 'min_samples_split': 3}    0.8309932006942402   16.628   62      
{'criterion': 'entropy', 'max_depth': 21.0, 'max_features': 24, 'min_samples_split': 2}    0.8293261843483514   21.899   63      
{'criterion': 'gini', 'max_depth': 24.0, 'max_features': 5, 'min_samples_split': 7}        0.8262085018823836   12.568   64      
{'criterion': 'gini', 'max_depth': 30.0, 'max_features': 7, 'min_samples_split': 3}        0.8271759636324952   14.4     65      
{'criterion': 'entropy', 'max_depth': 29.0, 'max_features': 7, 'min_samples_split': 5}     0.8347431224601489   15.675   66      
{'criterion': 'gini', 'max_depth': 26.0, 'max_features': 21, 'min_samples_split': 2}       0.8286538140201337   21.689   67      
{'criterion': 'entropy', 'max_depth': 17.0, 'max_features': 16, 'min_samples_split': 6}    0.8287498776191907   21.561   68      
{'criterion': 'entropy', 'max_depth': 23.0, 'max_features': 17, 'min_samples_split': 5}   